# Load and transform TwoPattern dataset
http://timeseriesclassification.com/description.php?Dataset=TwoPatterns

## Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from time import time
from tqdm import tqdm

time_index = int(time())

DATA_DIR = "data_two_pattern"
DIFFERENT_LENS = True
UNBALANCED = True # if set to unbalanced, samples are shuffled so that data is unbalanced 

NO_TEST_SAMPLES = 800 # there are total 5000 samples
NO_TRAIN_SAMPLES = 200
if NO_TEST_SAMPLES + NO_TRAIN_SAMPLES > 5000:
    raise Exception

infix = "dif" if DIFFERENT_LENS else "eq"
infix2 = "unbal_" if UNBALANCED else ""
OUTPUT_FILENAME = f"two_pattern_{infix}_lens_{infix2}ntrain{NO_TRAIN_SAMPLES}.pkl"

## Prepare and save dataset

In [ ]:
converted_lines = []
for filename in ["test.txt", "train.txt"]:
    with open (f"{DATA_DIR}/{filename}", "r") as f:
        lines = f.readlines()
        for l in lines:
            floats = list(map(float, l.split()))
            converted_lines.append(floats)

sample_ids = list(range(len(converted_lines)))
true_labels = [int(X[0])-1 for X in converted_lines] # labels should start from 0
all_X_samples = [np.array(X[1:]).reshape(-1,1) for X in converted_lines]

In [ ]:
np.array(all_X_samples).flatten().min()

In [ ]:
all_X_samples[5].shape

In [ ]:
# if different lenghts, cut some of samples
if DIFFERENT_LENS:
    new_samples = []
    for X in all_X_samples:
        cut = np.random.randint(1,40)
        new = X[:-cut]
        new_samples.append(new)
        if len(new) == 0:
            raise Exception
    all_X_samples = new_samples # overwrite sample list

sample_lengths = [sample.shape[0] for sample in all_X_samples]

labels_df = pd.DataFrame({
    "sample_id": sample_ids,
    "true_label": true_labels,
    "sample_len": sample_lengths
})

In [ ]:
labels_df.head()

In [ ]:
test_samples_ids = []
train_samples_ids = []
for label, sub_df in labels_df.groupby("true_label"):
    if UNBALANCED:
        frac = [0.1, 0.1, 0.2, 0.6]
        train_ids = sub_df.sample(int(np.ceil(frac[label]*NO_TRAIN_SAMPLES))).index.values
    else:
        train_ids = sub_df.sample(int(NO_TRAIN_SAMPLES/4)).index.values
    
    test_ids = sub_df.drop(train_ids).sample(int(NO_TEST_SAMPLES/4)).index.values
    print(f"For {label = } {len(train_ids) = }, {len(test_ids) = }")
    
    train_samples_ids.extend(train_ids)
    test_samples_ids.extend(test_ids)
    
indices_splits_lst = [{"test_samples_ids": test_samples_ids, "train_samples_ids": train_samples_ids}]

In [ ]:
output_path = f"{DATA_DIR}/{OUTPUT_FILENAME}_{time_index}"

metadata = {
    "NO_TRAIN_SAMPLES": len(train_samples_ids),
    "NO_TEST_SAMPLES": len(test_samples_ids),
    "MAX_SAMPLE_LEN": max(sample_lengths),
    "MIN_SAMPLE_LEN": min(sample_lengths)
}

data = {
    'generating_model': "SIM_TwoPat",
    'data_filename': output_path,
    'time_index': time_index,
    'models_lst': None,
    'labels_df': labels_df,
    'all_X_samples': all_X_samples,
    'indices_splits_lst': indices_splits_lst,
    'metadata': metadata,
    'description': "http://timeseriesclassification.com/description.php?Dataset=TwoPatterns"
}

with open(output_path, 'wb') as f:
    pickle.dump(data, f)
    print(f"Data saved to {output_path}.")

## EDA

### Is dataset balanced?

In [ ]:
labels_df.set_index("sample_id").iloc[train_samples_ids]["true_label"].value_counts()

In [ ]:
labels_df.set_index("sample_id").iloc[test_samples_ids]["true_label"].value_counts()

In [ ]:
labels_df.set_index("sample_id").iloc[test_samples_ids]

In [ ]:
all_X_samples[0].shape

### Plots of samples

In [ ]:
if not UNBALANCED and not DIFFERENT_LENS: # create plots only for balanced data (cant plot easily different lengths)
    labels = ["DD", "UD", "DU", "UU"]
    plt.rcParams["figure.figsize"] = (40, 16)
    plt.rcParams['font.size'] = 30
    fig, axes = plt.subplots(2,2, sharey="all")
    plt.suptitle(f"Two samples of each class")
    for i, df in tqdm(labels_df.groupby("true_label"), desc = "Preparing plots"):
        ax = axes.flat[i]
        ids = df["sample_id"].sample(2).values
        samples = np.array([all_X_samples[id] for id in ids])
        ax.plot(samples.reshape((2, -1)).T, lw = 3)
        ax.set_title(f"({labels[i]}) label = {i}")
        ax.grid()
        
    plt.savefig(f"./plots/models_twoPat_{time_index}.jpg")
    plt.show()